# LOAD ALL THE THINGS COMPUTED FROM NOTEBOOK 03

In [2]:
import pickle
with open("models/model_artifacts.pkl", "rb") as f:
    artifacts = pickle.load(f)

selected_vars = artifacts["selected_vars"]
woe_tables = artifacts["woe_tables"]
bin_edges_dict = artifacts["bin_edges_dict"]

In [3]:
import pandas as pd
import numpy as np

In [4]:
train_model_base=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\train_model_base.csv")

# TRANSFORM VARIABLES TO THEIR WOEs

In [9]:
#checking how woe table looks like
woe_tables[selected_vars[0]]

EXT_SOURCE_3_BIN
(-0.000473, 0.257]   -0.969891
(0.257, 0.371]       -0.369300
(0.371, 0.46]        -0.065417
(0.46, 0.537]        -0.037298
(0.537, 0.606]        0.448722
(0.606, 0.669]        0.611140
(0.669, 0.735]        0.796312
(0.735, 0.896]        0.956332
Name: WOE, dtype: float64

In [12]:
#create separate function for numerical variables
def transform_numeric_to_woe(dataframe,variable):
    bin_edges=bin_edges_dict[variable]
    binned_series=pd.cut(dataframe[variable],bins=bin_edges,include_lowest=True)
    woe_mapping=woe_tables[variable]['WOE']
    transformed_series=binned_series.map(woe_mapping)
    return transformed_series

In [14]:
EXT_SOURCE_2_TRANSFORMED=transform_numeric_to_woe(train_model_base,"EXT_SOURCE_2")
EXT_SOURCE_2_TRANSFORMED.value_counts(dropna=False)

EXT_SOURCE_2
 0.079621    24862
-0.939145    24610
 0.549551    24604
-0.046149    24602
-0.228828    24602
 1.064822    24601
 0.342355    24600
 0.694813    24598
-0.451157    24592
 0.224187    24337
Name: count, dtype: int64

In [17]:
#FUNCTION FOR CATEGORICAL VARIABLE
def transform_categorical_to_woe(dataframe,variable):
    woe_mapping=woe_tables[variable]['WOE']
    transformed_series=dataframe[variable].map(woe_mapping).fillna(0)
    return transformed_series

# TEST THE ABOVE FUNCTION ON TEST_DATA

In [15]:
#import test data
x_test_clean=pd.read_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_test_clean.csv")

In [16]:
EXT_SOURCE_2_TRANSFORMED=transform_numeric_to_woe(x_test_clean,"EXT_SOURCE_2")
EXT_SOURCE_2_TRANSFORMED.value_counts(dropna=False)

EXT_SOURCE_2
-0.939145    6221
 0.079621    6213
 0.549551    6193
-0.228828    6186
 0.342355    6181
-0.046149    6159
-0.451157    6158
 0.224187    6108
 1.064822    6060
 0.694813    6024
Name: count, dtype: int64

# BULK TRANSFORMATION WOE

In [20]:
#create 2 dataframes 
x_train_woe=pd.DataFrame()
x_test_woe=pd.DataFrame()

In [21]:
train_model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246008 entries, 0 to 246007
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   EXT_SOURCE_3         246008 non-null  float64
 1   EXT_SOURCE_2         246008 non-null  float64
 2   EXT_SOURCE_1         246008 non-null  float64
 3   DAYS_EMPLOYED        246008 non-null  int64  
 4   AMT_GOODS_PRICE      246008 non-null  float64
 5   OCCUPATION_TYPE      246008 non-null  object 
 6   DAYS_BIRTH           246008 non-null  int64  
 7   ORGANIZATION_TYPE    246008 non-null  object 
 8   NAME_INCOME_TYPE     246008 non-null  object 
 9   NAME_EDUCATION_TYPE  246008 non-null  object 
 10  TARGET               246008 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 20.6+ MB


In [22]:
for col in selected_vars:
    if train_model_base[col].dtype=='object':
        x_train_woe[col]=transform_categorical_to_woe(train_model_base,col)
        x_test_woe[col]=transform_categorical_to_woe(x_test_clean,col)
    else:
        x_train_woe[col]=transform_numeric_to_woe(train_model_base,col)
        x_test_woe[col]=transform_numeric_to_woe(x_test_clean,col)

In [24]:
x_train_woe.head(15)

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,AMT_GOODS_PRICE,OCCUPATION_TYPE,DAYS_BIRTH,ORGANIZATION_TYPE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE
0,-0.037298,-0.228828,-0.049847,0.336116,0.045561,0.235234,0.533604,0.432406,0.434849,-0.114665
1,0.956332,0.079621,-0.049847,-0.367960,0.220342,-0.303906,0.421670,-0.434254,-0.191389,-0.114665
2,0.796312,0.549551,0.483362,0.336116,0.231366,0.235234,0.266501,0.432406,0.434849,-0.114665
3,-0.969891,-0.939145,-0.049847,-0.192015,-0.557737,0.311292,-0.339430,-0.150859,-0.191389,-0.114665
4,-0.037298,-0.046149,0.987788,-0.033212,-0.044244,0.235234,0.126310,-0.150859,-0.191389,-0.114665
5,-0.065417,0.079621,-0.049847,-0.192015,0.112657,0.235234,-0.397820,0.225674,-0.191389,-0.114665
6,-0.037298,-0.451157,-0.049847,0.336116,0.045561,0.235234,0.004733,0.432406,0.434849,-0.114665
7,0.796312,-0.228828,-0.049847,0.336116,0.045561,0.235234,0.421670,0.432406,0.434849,-0.114665
8,0.611140,0.224187,0.987788,0.336116,0.112657,0.235234,0.533604,0.432406,0.434849,-0.114665
9,-0.065417,-0.046149,0.987788,-0.286764,0.112657,0.311292,0.050225,-0.150859,-0.191389,0.454906


In [25]:
x_test_woe.head(15)

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,AMT_GOODS_PRICE,OCCUPATION_TYPE,DAYS_BIRTH,ORGANIZATION_TYPE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE
0,-0.369300,0.342355,-0.011893,-0.374438,-0.044244,0.281957,0.266501,0.331053,0.366365,0.454906
1,0.796312,0.079621,-0.011893,0.571947,-0.116798,-0.285927,0.126310,-0.028921,-0.191389,-0.114665
2,-0.065417,0.224187,-0.049847,0.301072,-0.044244,-0.303906,-0.094726,-0.150859,-0.191389,-0.114665
3,0.956332,1.064822,-0.049847,0.571947,-0.557737,0.311292,0.266501,0.062787,0.088176,-0.114665
4,-0.037298,0.079621,-0.611732,-0.192015,-0.044244,-0.303906,0.004733,-0.434254,-0.191389,-0.114665
5,0.611140,-0.451157,-0.011893,0.301072,0.231366,-0.303906,-0.094726,-0.434254,-0.191389,-0.114665
6,0.611140,0.549551,-0.049847,-0.367960,0.112657,0.535660,-0.094726,-0.270694,-0.191389,0.454906
7,-0.037298,-0.046149,-0.611732,-0.033212,-0.557737,0.281957,-0.397820,0.494069,-0.191389,-0.044546
8,0.796312,0.549551,-0.049847,0.301072,-0.557737,-0.198140,0.126310,-0.270694,-0.191389,-0.114665
9,-0.065417,-0.939145,-0.049847,-0.033212,-0.236840,0.311292,-0.094726,-0.028921,0.088176,0.454906


In [26]:
x_train_woe.isnull().sum().sum()

np.int64(0)

In [30]:
x_test_woe.isnull().sum().sum()

np.int64(0)

In [32]:
x_train_woe.shape

(246008, 10)

In [31]:
x_test_woe.shape

(61503, 10)

# EXPORT X_TRAIN_WOE AND X_TEST_WOE

In [33]:
x_train_woe.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_train_woe.csv",index=False)

In [34]:
x_test_woe.to_csv(r"E:\DV Data Science\Retail_PD_Scorecard\data\processed\x_test_woe.csv",index=False)